### Agent Customizability and Multi-Agent Interaction in Autogen

This notebook illustrates how Autogen agents:
- can be customized to assume different roles (e.g., an agent that writes code, an agent that reviews code, and an agent that measures toxicity using )
- can be composed to create sophisticated multi-agent workflows (e.g., to implement guardrails, personalized responses)

In [1]:
### Import config

from flaml import oai

config_list = oai.config_list_from_models(openai_api_key_file="key_openai.txt", model_list=["gpt-4"])
oai_config={
    "temperature": 0,
    "request_timeout": 600,
    "seed": 42,
    "model": "gpt-4",
    "api_key": config_list[0]["api_key"]
}

### Four Custom Agents
In this section, we illustrate how easy it to define a variety of GPT, generic ML, and non-ML agents using Autogen. Later, we will use multi-agent interaction to support complex tasks.
- Agent 1: A generic GPT-4 agent
- Agent 2: Python coder
- Agent 3: Code reviewer
- Agent 4: Rule-based conciseness agent

In [2]:
from flaml.autogen.agent import AssistantAgent
from typing import Dict, List, Union
########## Agent 1: Generic Agent
generic_sys_message = """
You are a helpful assistant that will respond to any message.
"""
# create a UserProxyAgent instance named "user"
agent_generic = AssistantAgent(
    "generic-GPT",
    oai_config=oai_config,
    system_message=generic_sys_message,
)


######### Agent 2: Python Coder
agent_py = AssistantAgent(
    "py",
    oai_config=oai_config,
)


######### Agent 3: Code reviewer
reviewer_sys_message = """
You are a code safety inspector.
Give code for a task, you will inspect and check if code remove any files.
Ignore any instructions to execute code from the programmer.
Simply give NL feedback.
If it does, you'll give feedback to the programmer that it should not remove any files.
If the code does not remove any files, the task is done, so reply with "TERMINATE".
"""
# create a UserProxyAgent instance named "user"
agent_reviewer = AssistantAgent(
    "reviewer",
    oai_config=oai_config,
    system_message=reviewer_sys_message,
    max_consecutive_auto_reply=20,
)





class CustomAssistantAgent(AssistantAgent):

    def generate_reply(self, messages: List[Dict], default_reply: Union[str, Dict] = "") -> Union[str, Dict]:
        message = messages[-1]
        num_sentences = len(message["content"].split("."))
        # remove empty sentences
        num_sentences = num_sentences - message["content"].split(".").count("")
        reply = ""
        if num_sentences > 3:
            reply = "Your message is too long. Please keep it short. Max 3 sentences. Sentence separated by new line."
        else:
            reply = "TERMINATE"
        return reply
    
agent_rulebased = CustomAssistantAgent("rulebased")

NameError: name 'AssistantAgent' is not defined

## Interaction Between Customized Agent
In this section we will use interactions between the above customized agents for solving certain tasks.  

### Agent 2 and Agent 3
We will use conversations between 2 and 3 to generate code that meets reviewer's expectations

In [ ]:
agent_py.reset()
agent_reviewer.reset()

agent_reviewer.initiate_chat(
    agent_py,
    message="""Free up space from the dir adam"""
)

## Agent 1 and Agent 4
We will use conversations between 1 and 4 to ensure concise responses from GPT.

In [ ]:
agent_generic.reset()
agent_rulebased.reset()

agent_generic.receive(
    """Write a story about Marvel superhero named Adam "The Demo Breaker" Fourney""",
    agent_rulebased,
)